# Exploring 501 records from 1543 (Kickback from Lex's R Code)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 15, 2021
***
`This Jupyter Notebook explores 501 records (of the 1543 OTHER records) that were returned after being ran through Lex's R Code. The process for exploring these records is as follows:`

1. Explore the 'DATASET' field, which indicates what type of Origin Location Reference appears within the 'REMARKS' field for each record (e.g., AIRPORT, NAVAID, AIRPORT DIRECTLY OVER, NAVAID DIRECTLY OVER, RUNWAY, FIX RADIAL, HELIPORT, etc) 
<br>

2. For each of the unique Origin Location References that appear in the 'DATASET' field:

    a. Run the uas_loc_parts() function of the dataframe 
    
    b. Link to the appropriate Origin Location Reference linking function (e.g., link_airportsCleaned_ident(), link_airportsCleaned_icao(), nav_link())
        i. Separate the field that were successfully linked with the Origin Location Reference Dataset(s) and those that were not. Those that were not linked successfully will be moved to the exceptions file for further review.
    c. Run the successfully linked dataframe through the uas_lat_long() function to calculate the UAS Latitude/Longitude coordinates for each respective record/remark. 

***
### INPUT: 
*Dataframe (read from a .csv file) containing 501 records, of 1543 records that did not hit on the initial 'XX NM dir XXX' regular expressions, and was not initially picked up by our Runways regular expressions.*

### OUTPUT:
<font color='red'>**EXCPETIONS FILE**</font>

*Designated Point: 1 record (no link to Designated Points)*

*FAA Airport Identifier: 38 records (no link to Airports or NAVAID Systems)*

<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*FAA Airport Identifier: 451 records*

*NAVAID System: 1 record*

*CEDAR Lat/Long: 10 records*



## Import Statements

In [2]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic
from fractions import Fraction

from IPython.display import display, HTML

In [3]:
output501 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace_All_Points_Standard_501OUTOF1072.csv')
output501

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASWEIGHTGT55.CEDAR,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET
0,11,Aircraft observed a red uas at UNKN altitude 1...,UNKN,2/17/2018,4:52,2018-02-17T04:52Z,DEN,UAS,NaN,LAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 NM E of LAX,AIRPORT
1,1689,"Aircraft reported auas 3 feet in diameter, 3 N...",UNKN,4/26/2018,21:43,2018-04-26T21:43Z,DEN,UAS,NaN,N90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM E of LGA,AIRPORT
2,71,Aircraft observed a light colored uas 2 feet i...,UNKN,7/14/2018,22:27,2018-07-14T22:27Z,DEN,UAS,NaN,P50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 NM S of PHX,AIRPORT
3,1885,"Aircraft reported a reddish uas, 12 inches in ...",UNKN,11/11/2018,20:58,2018-11-11T20:58Z,NCRCC,UAS,NaN,PCT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM S FDK,AIRPORT
4,1930,"Aircraft Wbound at 4,000 feet6 NM NW of MIA. A...",UNKN,2/1/2019,3:28,2019-02-01T03:28Z,DEN,UAS,NaN,MIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 NM NW of MIA,AIRPORT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,1536,"Aircraft observed a uas while W bound at 1,400...",UNKN,8/17/2021,17:09,2021-08-17T17:09Z,DEN,UAS,SUSPICIOUS ACTIVITY,EWR,...,No,UAS Dimensions (feet): NA,NaN,No,No,"UAL967, B757 2 MILE RIGHT BASE RY 29 REPORTED ...","Facility summary, weather and falcon bookmark ...",NaN,3 NM E of EWR,AIRPORT
497,1537,Aircraft observed a red top uas off the right ...,UNKN,8/18/2021,22:41,2021-08-18T22:41Z,DEN,UAS,NaN,RDU,...,No,UAS Dimensions (feet): NA,NaN,No,No,"N514RR, PRM1, ON THE BASE FOR RY23R AT 4000 FE...","Facility summary, weather and falcon bookmark ...",NaN,NaN,LAT/LONG
498,1540,Aircraft observed a uas off the right side whi...,UNKN,8/21/2021,15:40,2021-08-21T15:40Z,DEN,UAS,SUSPICIOUS ACTIVITY,PHX,...,No,UAS Dimensions (feet): UNK,Rotor Craft,No,No,"SCX603, B737, 6 MILE FINAL AT 3400' FEET, REPO...",QA reviewed.,NaN,6 NM W of PHX,AIRPORT
499,1541,Aircraft observed a uas below the aircraft whi...,UNKN,8/21/2021,16:30,2021-08-21T16:30Z,DEN,UAS,NaN,L30,...,No,UAS Dimensions (feet): NA,Rotor Craft,No,No,"At 1615Z, N850KS reported a drone 100 feet bel...",Reviewed MOR Summary for a UAS report. P Rober...,NaN,3 NM N of VGT,AIRPORT


In [4]:
output501_df = output501[['REMARKS', 'DATASET', 'UASLOCATION']]
output501_df

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a red uas at UNKN altitude 1...,AIRPORT,12 NM E of LAX
1,"Aircraft reported auas 3 feet in diameter, 3 N...",AIRPORT,3 NM E of LGA
2,Aircraft observed a light colored uas 2 feet i...,AIRPORT,7 NM S of PHX
3,"Aircraft reported a reddish uas, 12 inches in ...",AIRPORT,3 NM S FDK
4,"Aircraft Wbound at 4,000 feet6 NM NW of MIA. A...",AIRPORT,6 NM NW of MIA
...,...,...,...
496,"Aircraft observed a uas while W bound at 1,400...",AIRPORT,3 NM E of EWR
497,Aircraft observed a red top uas off the right ...,LAT/LONG,NaN
498,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM W of PHX
499,Aircraft observed a uas below the aircraft whi...,AIRPORT,3 NM N of VGT


In [5]:
output501_df['DATASET'].unique()

array(['AIRPORT', 'DESIGNATED POINT', 'NAVAID', 'LAT/LONG'], dtype=object)

# LAT/LONG (CEDAR)

`These 10 remarks contained the UAS Lat/Long information in the CEDAR Remarks`

In [9]:
latlong10 = output501[output501['DATASET'] == 'LAT/LONG'].reset_index(drop=True)[['REMARKS', 'UASLOCATION', 'UASLAT.CEDAR', 'UASLONG.CEDAR']]
latlong10

,REMARKS,UASLOCATION,UASLAT.CEDAR,UASLONG.CEDAR
0,Aircraft observed a uas off the left side 300 ...,NaN,42.592400,78.365700
1,Aircraft observed a uas off the 2 o'clock posi...,NaN,37.816660,-122.475830
2,Aircraft observed a uas off the 12 o'clock sid...,NaN,41.592100,87.414500
3,Aircraft observed a uas off the right side whi...,NaN,31.110000,79.250000
4,Aircraft observed a Quadcopter while N Wbound ...,NaN,39.190556,-76.673333
5,Aircraft observed a uas off the left side whil...,NaN,37.170499,-79.734833
6,"Aircraft observed a white uas while at 1,500 f...",NaN,37.634300,-122.362200
7,Aircraft observed a grey quadcopter below whil...,NaN,39.151100,-76.267700
8,Aircraft observed a red top uas off the right ...,NaN,35.524800,78.322900
9,Aircraft observed a uas side while SE bound at...,NaN,38.718300,-121.257100


In [10]:
# Export to .csv file 
latlong10.to_csv('10LLof501_uasLatLong.csv', index=False)

# AIRPORT

In [20]:
airport489 = output501_df[output501_df['DATASET'] == 'AIRPORT'].reset_index(drop=True)
airport489

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a red uas at UNKN altitude 1...,AIRPORT,12 NM E of LAX
1,"Aircraft reported auas 3 feet in diameter, 3 N...",AIRPORT,3 NM E of LGA
2,Aircraft observed a light colored uas 2 feet i...,AIRPORT,7 NM S of PHX
3,"Aircraft reported a reddish uas, 12 inches in ...",AIRPORT,3 NM S FDK
4,"Aircraft Wbound at 4,000 feet6 NM NW of MIA. A...",AIRPORT,6 NM NW of MIA
...,...,...,...
484,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM N of CDW
485,Aircraft observed a uas while conducting patte...,AIRPORT,.5 NM S of AFW
486,"Aircraft observed a uas while W bound at 1,400...",AIRPORT,3 NM E of EWR
487,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM W of PHX


In [21]:
def link_airports (dataframe):
    '''
    This function links the dataframe input by the user to the cleaned airports_cleaned.csv dataset on the
    'IDENT' field.
    
    input : dataframe is a dataframe with a field called 'UASLOCATION' that includes an FAA Airport Identifier
    output : the same dataframe with relative airport information appended to each row
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    # Create IDENT field in dataframe
    ident_list = []
    
    # Grab FAA Airport Identifier from the UASLOCATION field
    for i in range(len(dataframe)):
        ident = dataframe['UASLOCATION'].loc[i].split()[-1]
        ident_list.append(ident)
    
    # Create IDENT field in dataframe
    dataframe['IDENT'] = ident_list
    
    # Merge dataframe with airportsC df and drop records for which there is no matching airport FAA IDENTIFIER
    outputdf = pd.merge(dataframe, airportsC, on = 'IDENT', how = 'left').dropna()
    
    return outputdf

In [60]:
airports451 = link_airports(airport489).reset_index(drop=True)
#airports451

In [38]:
# Create Dictionary of Bearings and Respective Degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }

uas_loc = airports451['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]

    if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')

    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')

    distances.append(distance_nm)
    bearings.append(bearing_abrev)
    
for i in range(len(distances)):
    if '/' in distances[i]:
        x = Fraction(distances[i])
        distances[i] = float(x)
    else:
        distances[i] = float(distances[i])

airports451['Distance_NM'] = distances
airports451['Bearing'] = bearings

# Convert all distance from NM to kilometers and all bearing abbreviations to their associated degrees
dist_kilo = []

for i in range(len(airports451)):
    distanceKilo = airports451['Distance_NM'][i] * 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(airports451['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

for i in range(len(bearDegree)):
    if bearDegree['Bearing'][i] == '':
        bearDegree.replace('', np.nan, inplace = True)

airports451['Distance_Kilometers'] = dist_kilo
airports451['Bearing_Degrees'] = bearDegree

uas_lat = []
uas_long = []


for i in range(len(airports451)):
    lat_airport = pd.to_numeric(airports451['LATITUDE'][i])
    long_airport = pd.to_numeric(airports451['LONGITUDE'][i])
    b = pd.to_numeric(airports451['Bearing_Degrees'][i])
    d = pd.to_numeric(airports451['Distance_Kilometers'][i])

    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
airports451['UAS_Latitude'] = uas_lat
airports451['UAS_Longitude'] = uas_long

In [39]:
airports451

,REMARKS,DATASET,UASLOCATION,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,Distance_NM,Bearing,Distance_Kilometers,Bearing_Degrees,UAS_Latitude,UAS_Longitude
0,Aircraft observed a red uas at UNKN altitude 1...,AIRPORT,12 NM E of LAX,LAX,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,Los Angeles Intl,33.942496,-118.408049,KLAX,12.0,E,22.224,90,33.942262,-118.167652
1,"Aircraft reported auas 3 feet in diameter, 3 N...",AIRPORT,3 NM E of LGA,LGA,E866CAA3-781C-4162-8C31-D16CC9E3E92E,Laguardia,40.777250,-73.872611,KLGA,3.0,E,5.556,90,40.777231,-73.806796
2,Aircraft observed a light colored uas 2 feet i...,AIRPORT,7 NM S of PHX,PHX,639E89B2-F2AE-4589-BC8E-2694490CE138,Phoenix Sky Harbor Intl,33.434278,-112.011583,KPHX,7.0,S,12.964,180,33.317391,-112.011583
3,"Aircraft reported a reddish uas, 12 inches in ...",AIRPORT,3 NM S FDK,FDK,F7C1C653-CD99-47B2-B41B-C84410D9BD04,Frederick Muni,39.417568,-77.374297,KFDK,3.0,S,5.556,180,39.367525,-77.374297
4,"Aircraft Wbound at 4,000 feet6 NM NW of MIA. A...",AIRPORT,6 NM NW of MIA,MIA,28A25B5E-8EB8-4F2F-9D2B-3605D2654733,Miami Intl,25.795361,-80.290116,KMIA,6.0,NW,11.112,315,25.866264,-80.368509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM N of CDW,CDW,8B84CC8D-6072-44DF-B7B9-EC41CFF4CA36,Essex County,40.875222,-74.281361,KCDW,6.0,N,11.112,0,40.975283,-74.281361
447,Aircraft observed a uas while conducting patte...,AIRPORT,.5 NM S of AFW,AFW,8F919C0A-6FB5-4E2F-8B36-843AA0C58C60,Fort Worth Alliance,32.990386,-97.319441,KAFW,0.5,S,0.926,180,32.982036,-97.319441
448,"Aircraft observed a uas while W bound at 1,400...",AIRPORT,3 NM E of EWR,EWR,6AB3CFCC-F6EE-4B71-A201-2DCA9BB7F2F1,Newark Liberty Intl,40.692480,-74.168687,KEWR,3.0,E,5.556,90,40.692461,-74.102955
449,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM W of PHX,PHX,639E89B2-F2AE-4589-BC8E-2694490CE138,Phoenix Sky Harbor Intl,33.434278,-112.011583,KPHX,6.0,W,11.112,270,33.434220,-112.131077


In [40]:
# Export to .csv file 
airports451.to_csv('air451_uasLatLong.csv', index=False)

`Find 38 records for which there was no actual link to the airports_cleaned.csv dataset via the IDENT field`

In [59]:
air_merge = df_all = airport489.merge(airports451.drop_duplicates(), on=['REMARKS'], 
                   how='left', indicator=True)
#air_merge

In [58]:
air_unid38 = air_merge[air_merge['_merge'] == 'left_only'].reset_index(drop=True)
#air_unid38

In [57]:
air_unid38 = air_unid38.rename(columns = {'IDENT_x' : 'IDENT', 'DATASET_x' : 'DATASET', 'UASLOCATION_x' : 'UASLOCATION'})
#air_unid38

In [55]:
airunid38_df = air_unid38[['REMARKS', 'DATASET', 'UASLOCATION', 'IDENT']]
airunid38_df

,REMARKS,DATASET,UASLOCATION,IDENT
0,Aircraft reported a black uas from the left si...,AIRPORT,10 NM E of VHP,VHP
1,Aircraft observed a uas pass over the aircraft...,AIRPORT,14 NM NW of GEP,GEP
2,"Aircraft observed a silver, fixed wing uas 2,0...",AIRPORT,19 NM E of GZG,GZG
3,Aircraft observed a uas while Ebound climbing ...,AIRPORT,6 NM S of RZS,RZS
4,"Aircraft observed a uas operating at 6,500 fee...",AIRPORT,4 NM S of CRI,CRI
5,Aircraft observed a black or dark gray uas pas...,AIRPORT,21 NM S of PGO,PGO
6,Aircraft observed a rotor wing uas beneath the...,AIRPORT,5 NM SE of OOM,OOM
7,"Aircraft observed a uas while Ebound at 5,000 ...",AIRPORT,5 NM W of HAR,HAR
8,Received report via MOR: Aircraft observed a r...,AIRPORT,1 NM NW of OBK,OBK
9,Aircraft observed a tiny uas possibly opposite...,AIRPORT,36 NM E of BDF,BDF


In [56]:
# Export to .csv file 
airunid38_df.to_csv('air38_EXCEPTIONS.csv', index=False)

# NAVAID System

In [62]:
nav1 = output501_df[output501_df['DATASET'] == 'NAVAID'].reset_index(drop=True)
nav1

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a small uas 100 feet below w...,NAVAID,3 NM N of SAF


In [64]:
def uas_loc_parts(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    ident_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get IDENT from UASLOCATION
        ident = uasLoc_col[i].split()[-1].replace('of', '')
        ident_list.append(ident)
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        bearing = re.sub('(NM|of|OF)', ' ', uasLoc_col[i]).split()[-2]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['IDENT'] = ident_list
    
    
    return dataframe

In [68]:
nav1_df = uas_loc_parts(nav1)
nav1_df

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,Aircraft observed a small uas 100 feet below w...,NAVAID,3 NM N of SAF,3,5.556,N,0,SAF


`Create function that links a dataframe to the NAVAID_cleaned.csv dataset on the 'IDENT' field`

In [66]:
def nav_link (dataframe):
    '''
    This functions merges the dataframe input by the user with the cleaned NAVAID_cleaned.csv dataset
    '''
    
    # Read in Cleaned NAVAID Systems dataset
    nav_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')
    
    output_df = pd.merge(dataframe, nav_clean, on = 'IDENT', how = 'left').reset_index(drop=True)
    
    return output_df

`Create function that calculates the UAS Latitude/Longitude Coordinates`

In [70]:
def uas_lat_long(dataframe):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param dataframe : The dataframe that includes the fields 'LATITUDE', 'LONGITUDE', 'Bearing_Degrees', 
                        'Distance_Kilometers' to have UAS Lat/Long values calculated for
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    uas_lat = []
    uas_long = []
    
    for i in range(len(dataframe)):
        lat_airport = pd.to_numeric(dataframe['LATITUDE'][i])
        long_airport = pd.to_numeric(dataframe['LONGITUDE'][i])
        b = pd.to_numeric(dataframe['Bearing_Degrees'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2 = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
        
    
    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    #dataframe.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return dataframe

In [72]:
nav1_latlong = uas_lat_long(nav_link(nav1_df))
nav1_latlong

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,UAS_Latitude,UAS_Longitude
0,Aircraft observed a small uas 100 feet below w...,NAVAID,3 NM N of SAF,3,5.556,N,0,SAF,-106.064946,35.540547,SANTA FE,35.590623,-106.064946


In [73]:
# Export to .csv file 
nav1_latlong.to_csv('nav1_uasLatLong.csv', index=False)

# DESIGNATED POINT

In [76]:
dp1 = output501_df[output501_df['DATASET'] == 'DESIGNATED POINT'].reset_index(drop=True)
dp1

,REMARKS,DATASET,UASLOCATION
0,Aircraft reported a red uas Nbound while the a...,DESIGNATED POINT,6 NM S COYLE


`Create function that links a dataframe to the designated_points_cleaned.csv dataset on the 'IDENT' field`

In [77]:
def dp_link (dataframe):
    '''
    This functions merges the dataframe input by the user with the cleaned NAVAID_cleaned.csv dataset
    '''
    
    # Read in Cleaned NAVAID Systems dataset
    dp_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/designated_points_cleaned.csv')
    
    output_df = pd.merge(dataframe, dp_clean, on = 'IDENT', how = 'left').reset_index(drop=True)
    
    return output_df

In [79]:
dp_link(uas_loc_parts(dp1))

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,Unnamed: 0,LATITUDE,LONGITUDE
0,Aircraft reported a red uas Nbound while the a...,DESIGNATED POINT,6 NM S COYLE,6,11.112,S,180,COYLE,NaN,NaN,NaN


In [80]:
# Export to .csv file 
dp1.to_csv('dp1_EXCEPTIONS.csv', index=False)